In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from connect_to_database import get_data_from_db

We are gathering data for 30 legendary rock artists in a cloud database (you can find more information about the E.T.L process here; https://github.com/VangelisChocholis/ETL_Spotify_data). The ultimate objective is to create a Streamlit web application that provides real-time visualizations of popularity and trends based on the collected data. Before proceeding with the construction of the web app, it is crucial to conduct Exploratory Data Analysis (EDA).

Read the data (use `get_data_from_db` to get data instead of pd.read_csv)

In [28]:
# read static data
artists_table = pd.read_csv('artists_table.csv')
albums_table = pd.read_csv('albums_table.csv')
tracks_table = pd.read_csv('tracks_table.csv')
tracks_faeaures_table = pd.read_csv('tracks_features_table.csv')
# read dynamic data
tracks_popularity_table = pd.read_csv('tracks_popularity_table.csv', parse_dates=['date'])
albums_popularity_table = pd.read_csv('albums_popularity_table.csv', parse_dates=['date'])
artists_popularity_table = pd.read_csv('artists_popularity_table.csv', parse_dates=['date'])
artists_followers_table = pd.read_csv('artists_followers_table.csv', parse_dates=['date'])

In [ ]:
# merge to get tracks data
data = pd.merge(tracks_table, albums_table, on='album_id')
data = pd.merge(data, artists_table, on='artist_id')
# get the last date of popularity
data = pd.merge(data, tracks_popularity_table.query('date == @tracks_popularity_table.date.max()'), on='track_id')
data = pd.merge(data, tracks_faeaures_table, on='track_id')


#artists = pd.merge(artists_table, artists_popularity_table, on='artist_id')
#artists = pd.merge(artists, artists_followers_table[['artist_id', 'followers']], on='artist_id')

In [32]:
data.columns

Index(['track_id', 'album_id', 'track_name', 'track_duration_ms',
       'track_spotify_url', 'track_preview_url', 'track_duration_display',
       'original_track_name', 'artist_id', 'album_name', 'album_release_date',
       'album_total_tracks', 'album_image_large', 'album_image_medium',
       'album_image_small', 'original_album_name', 'artist_name', 'date',
       'track_popularity', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature', 'danceability', 'energy', 'track_key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'type'],
      dtype='object')

In [33]:
# drop unnecessary columns
data = data.drop(['track_spotify_url', 'track_preview_url', 'album_image_large', 
                  'album_image_medium', 'album_image_small',
                  'uri', 'analysis_url', 'track_href', 'duration_ms'], axis=1)

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5956 entries, 0 to 5955
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   track_id                5956 non-null   object        
 1   album_id                5956 non-null   object        
 2   track_name              5956 non-null   object        
 3   track_duration_ms       5956 non-null   int64         
 4   track_duration_display  5956 non-null   object        
 5   original_track_name     5956 non-null   object        
 6   artist_id               5956 non-null   object        
 7   album_name              5956 non-null   object        
 8   album_release_date      5956 non-null   int64         
 9   album_total_tracks      5956 non-null   int64         
 10  original_album_name     5956 non-null   object        
 11  artist_name             5956 non-null   object        
 12  date                    5956 non-null   datetime

In [40]:
# check for missing values
data.isna().sum(axis=0)

track_id                  0
album_id                  0
track_name                0
track_duration_ms         0
track_duration_display    0
original_track_name       0
artist_id                 0
album_name                0
album_release_date        0
album_total_tracks        0
original_album_name       0
artist_name               0
date                      0
track_popularity          0
time_signature            0
danceability              0
energy                    0
track_key                 0
loudness                  0
mode                      0
speechiness               0
acousticness              0
instrumentalness          0
liveness                  0
valence                   0
tempo                     0
type                      0
dtype: int64

In [45]:
# convert duration to seconds
data['track_duration_sec'] = data['track_duration_ms']/1000
data = data.drop('track_duration_ms', axis=1)

In [46]:
data.describe()

,album_release_date,album_total_tracks,date,track_popularity,time_signature,danceability,energy,track_key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_duration_sec
count,5956.000000,5956.000000,5956,5956.000000,5956.000000,5956.000000,5956.000000,5956.000000,5956.000000,5956.000000,5956.000000,5956.000000,5956.000000,5956.000000,5956.000000,5956.000000,5956.000000
mean,1988.912021,16.477166,2024-01-10 00:00:00.000000256,35.034251,3.892713,0.493097,0.634537,5.251007,-9.485259,0.739590,0.061575,0.261318,0.142479,0.214232,0.528509,122.277366,236.079101
min,1962.000000,5.000000,2024-01-10 00:00:00,0.000000,0.000000,0.000000,0.002940,0.000000,-38.575000,0.000000,0.000000,0.000002,0.000000,0.000000,0.000000,0.000000,4.066000
25%,1973.000000,11.000000,2024-01-10 00:00:00,23.000000,4.000000,0.391750,0.460750,2.000000,-11.621250,0.000000,0.031900,0.014500,0.000021,0.098300,0.330000,101.200500,175.333000
50%,1986.000000,13.000000,2024-01-10 00:00:00,34.000000,4.000000,0.498000,0.671000,5.000000,-8.709000,1.000000,0.039600,0.127000,0.002280,0.142000,0.532000,121.355500,227.526500
75%,2003.250000,19.000000,2024-01-10 00:00:00,46.000000,4.000000,0.597250,0.843000,9.000000,-6.222250,1.000000,0.058300,0.460250,0.113000,0.284000,0.733000,139.361500,280.478500
max,2023.000000,45.000000,2024-01-10 00:00:00,90.000000,5.000000,0.939000,1.000000,11.000000,-0.251000,1.000000,0.959000,0.994000,0.999000,0.993000,0.982000,216.304000,1758.306000
std,18.075910,8.602598,NaN,16.676660,0.419596,0.146110,0.243125,3.517015,4.676350,0.438895,0.084681,0.292752,0.269505,0.179638,0.252622,28.980473,103.789258


Notable observations:
- `album_total_tracks`: the max number is 45, which is quite large

- `track_popularity`: 75% of tracks have a popularity score below 46, with only 25% achieving some level of popularity.

- `time_signature`: The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). The time signature ranges from 3 to 7 indicating time signatures of "3/4", to "7/4". We have minumum of 0 which is invalid. More than 75% of tracks has time signature "4/4",  something to be expected for the rock genre.